In [1]:
import h5py
import numpy as np
from progressbar import ProgressBar

from work.environment import FEATURES_DATASET_FILE, OUTPUTS_DATASET_FILE

f_features = h5py.File(FEATURES_DATASET_FILE, 'r')
f_output = h5py.File(OUTPUTS_DATASET_FILE, 'r')

In [7]:
subsets = ('training', 'validation',)

f_file = h5py.File('./training_data_prediction.hdf5', 'w')


for subset in subsets:
    nb_clips = np.array([f_features[subset][key].shape[0] for key in f_features[subset].keys()], dtype=np.float32)
    nb_sequences = np.ceil(nb_clips / 20)
    total_nb_sequences = np.sum(nb_sequences).astype(np.int64)
    
    f_dataset = f_file.create_group(subset)
    input_features = np.zeros((total_nb_sequences, 20, 4096))
    classification_output = np.zeros((total_nb_sequences, 201))
    detection_output = np.zeros((total_nb_sequences, 20, 1))
    
    progbar = ProgressBar(max_value=total_nb_sequences)
    pos = 0
    progbar.update(pos)
    for k in f_features[subset].keys():
        features = f_features[subset][k]
        output = f_output[subset][k]
        
        nb_sequences, input_, classification, detection = get_input_and_output_for_video(features, output)
        input_features[pos:pos+nb_sequences,:,:] = input_
        classification_output[pos:pos+nb_sequences,:] = classification
        detection_output[pos:pos+nb_sequences,:,:] = detection
        
        pos += nb_sequences
        progbar.update(pos)
    progbar.finish()
        
    total_length = int(total_nb_sequences)//16 * 16
    f_dataset.create_dataset('input_features', data=input_features[:total_length,:,:], chunks=(4, 20, 4096), dtype='float32')
    f_dataset.create_dataset('classification_output', data=classification_output[:total_length,:], chunks=(256, 201), dtype='float32')
    f_dataset.create_dataset('detection_output', data=detection_output[:total_length,:,:], chunks=(256, 20, 1), dtype='float32')
    if subset == 'training':
        sample_weights = np.ones(input_features.shape[:2])
        sample_weights[np.all(input_features == 0, axis=2)] = 0
        f_dataset.create_dataset('sample_weights', data=sample_weights[:total_length,:], chunks=(256,20), dtype='float32')
    

100% (107361 of 107361) |##################| Elapsed Time: 0:20:05 Time: 0:20:05
100% (53273 of 53273) |####################| Elapsed Time: 0:10:52 Time: 0:10:52


In [4]:
import numpy as np

def get_input_and_output_for_video(features, output):
    timesteps = 20
    nb_classes = 201
    nb_clips = int(features.shape[0])
    assert output.shape[0] == nb_clips
    if nb_clips < timesteps:
        features_new = np.zeros((timesteps, 4096))
        features_new[:nb_clips,:] = features
        features = features_new
        output_new = np.zeros((timesteps,))
        output_new[:nb_clips] = output
        output = output_new
        nb_clips = int(features.shape[0])
    
    nb_sequences = np.ceil(nb_clips/float(timesteps)).astype(np.int32)
    
    input_ = np.zeros((nb_sequences, timesteps, 4096))
    classification = np.zeros((nb_sequences, nb_classes))
    detection = np.zeros((nb_sequences, timesteps, 1))
    
    for i in range(nb_sequences):
        if (i+1)*timesteps <= nb_clips:
            start_index, end_index = i*timesteps, (i+1)*timesteps
        else:
            assert (i+1) == nb_sequences
            start_index, end_index = nb_clips-timesteps, nb_clips
        input_[i,:,:] = features[start_index:end_index,:]
    
        seq_output = output[start_index:end_index]
        detection[i,:,0][seq_output[:] != 0] = 1
        activity_index = np.max(seq_output).astype(np.int32)
        classification[i, activity_index] = 1
        
        
    
    return nb_sequences, input_, classification, detection

In [5]:
def test_get_input_and_output_for_video():
    test_input = np.random.random((5, 4096))
    test_output = np.ones((5,))*5
    nb_sequences, input_, classification, detection = get_input_and_output_for_video(test_input, test_output)
    
    assert nb_sequences == 1
    assert input_.shape[0] == 1
    assert np.all(input_[0,:5,:] == test_input)
    assert np.all(input_[0,5:,:] == 0)
    assert classification[0,5] == 1
    assert np.all(classification[0,np.arange(201) !=5] == 0)
    assert np.all(detection[0,:5] == 1)
    assert np.all(detection[0,5:] == 0)
    
    test_input = np.random.random((40, 4096))
    test_output = np.zeros((40,))
    test_output[18:25] = 20
    nb_sequences, input_, classification, detection = get_input_and_output_for_video(test_input, test_output)
    
    assert nb_sequences == 2
    assert input_.shape[0] == 2
    assert np.all(input_[0] == test_input[:20])
    assert np.all(input_[1] == test_input[20:])
    assert np.all(classification[:, 20] == 1)
    assert np.all(classification[:, np.arange(201)!=20] == 0)
    assert np.all(detection[0,:18] == 0)
    assert np.all(detection[0,18:] == 1)
    assert np.all(detection[1,:5] == 1)
    assert np.all(detection[1,5:] == 0)
    
    test_input = np.random.random((35, 4096))
    test_output = np.zeros((35,))
    test_output[8:15] = 20
    nb_sequences, input_, classification, detection = get_input_and_output_for_video(test_input, test_output)
    
    assert nb_sequences == 2
    assert input_.shape[0] == 2
    assert np.all(input_[0] == test_input[:20])
    assert np.all(input_[1] == test_input[15:])
    assert np.all(classification[0, 20] == 1)
    assert np.all(classification[0, np.arange(201)!=20] == 0)
    assert np.all(classification[1,0] == 1)
    assert np.all(classification[1, np.arange(1, 201)] == 0)
    assert np.all(detection[0,:8] == 0)
    assert np.all(detection[0,8:15] == 1)
    assert np.all(detection[1,:] == 0)
    
test_get_input_and_output_for_video()

In [ ]:
print('echo')

In [ ]:
f_file.close()

In [ ]:
import h5py
import numpy as np

f_file = h5py.File('./training_data_prediction.hdf5', 'r+')


In [ ]:
sample_weights = np.ones(f_file['training']['input_features'].shape[:2])
sample_weights[np.all(f_file['training']['input_features'][:,:,:] == 0, axis=2)] = 0

In [ ]:
f_file.keys()
a=_[0]

In [ ]:
f_file[a].keys()

In [14]:
np.sum(sample_weights==0).astype(np.float32)/sample_weights.size

0.00081780162256312816

In [16]:
f_file['validation']['input_features'].shape[0]//256

208

In [17]:
f_file.close()